In [1]:
import time
import numpy as np
import matplotlib.pyplot as plt

In [2]:
class DP:
    def __init__(self):
        # state
        self.v = np.zeros((21, 21))
        # policy
        self.pi = np.zeros((21, 21))
        # poisson prob
        self.prob_A = np.zeros((21, 21))
        self.prob_B = np.zeros((21, 21))
        for i in range(21):
            for j in range(21):
                self.prob_A[i, j] = self._poisson_calculator(i, 3) * self._poisson_calculator(j, 3)
                self.prob_B[i, j] = self._poisson_calculator(i, 4) * self._poisson_calculator(j, 2)
        # joint prob: p(s',r|s,a)
        self.P = {}
        
    @staticmethod
    def _poisson_calculator(n, Lambda):
        return Lambda ** n / np.math.factorial(n) * np.exp(-Lambda)
        
    def prob_cal(self):
        """
        process: 
            S_T -> move car -> rent car -> return car -> S_(T+1)
        function: 
            calculate the sum of prob
        save / output:
            p(s',r|s,a) -> P(S_A, S_B, action): dict{((S_A_next, S_B_next, reward)): joint prob}
        """
        
        start = time.time()
        print("Prob calculation...")
        for state_A in range(21):  # loop the state
            print("State " + str(state_A))
            for state_B in range(21): 
                for action in range(-5, 6):
                    temp = {}
                    if action <= state_A and -action <= state_B:
                        for rent_A in range(21): # loop lend and return at day
                            for rent_B in range(21):
                                for return_A in range(21):
                                    for return_B in range(21): # loop action at night  
                                        state_A_next = min(20, state_A - min(rent_A,
                                                                             state_A - action) + return_A - action)
                                        state_B_next = min(20, state_B - min(rent_B,
                                                                             state_B + action) + return_B + action)
                                        reward = 10 * min(rent_A, state_A - action) + \
                                                 10 * min(rent_B, state_B + action) - \
                                                 abs(action) * 2
                                        temp[((state_A_next, state_B_next),reward)] = temp.get(
                                            ((state_A_next, state_B_next),reward), 0)
                                        temp[((state_A_next, state_B_next),reward)] += \
                                            self.prob_A[rent_A, return_A] * self.prob_B[rent_B, return_B]
                        self.P[(state_A, state_B), action] = temp
        end = time.time()
        print(f"cost {round(end - start, 2)} s")
        
    def policy_eval(self, epsilon=0.1):
        print("Policy evaluation...")
        start = time.time()
        while True:
            Delta = 0
            for state_A in range(21):
                for state_B in range(21):
                    action = self.pi[state_A, state_B]
                    p = self.P[(state_A, state_B), action]
                    temp = 0
                    for keys, values in p.items():
                        (states, reward) = keys
                        possibility = values
                        temp += (reward + 0.9 * self.v[states]) * possibility
                    old_value = self.v[state_A, state_B]
                    self.v[state_A, state_B] = temp
                    Delta = max(Delta, abs(self.v[state_A, state_B] - old_value))
            if Delta < epsilon:
                break
        end = time.time()
        print(f"cost {round(end - start, 2)} s")
        
    def policy_improvement(self):
        start = time.time()
        print("Policy improvement...")
        policy_stable = True
        for state_A in range(21):
            for state_B in range(21):
                action_value = np.zeros(11)
                for action in range(-5, 6):
                    if action <= state_A and -action <= state_B:
                        p = self.P[(state_A, state_B), action]
                        for keys, values in p.items():
                            (states, reward) = keys
                            possibility = values
                            action_value[action + 5] += (reward + 0.9 * self.v[states]) * possibility
                    else:
                        action_value[action + 5] = -1e9
                old_action = self.pi[state_A, state_B]
                self.pi[state_A, state_B] = np.argmax(action_value) - 5
                if self.pi[state_A, state_B] != old_action:
                    policy_stable = False
        end = time.time()
        print(f"cost {round(end - start, 2)} s")
        return policy_stable
    
    def train(self):
        self.prob_cal()
        for q in range(5):
            print(f"***** Big loop {q + 1} *****")
            self.policy_eval()
            if self.policy_improvement():
                break

In [3]:
if __name__ == "__main__":
    dp = DP()
    dp.train()
    
    # plt.imshow(dp.pi, cmap=plt.cm.get_cmap('gray', 9), origin='lower')
    # plt.yticks(np.arange(0,21,5))
    # plt.colorbar()
    
    # plt.imshow(dp.v, cmap='gray', origin='lower')
    # plt.yticks(np.arange(0,21,5))
    # plt.colorbar()

Prob calculation...
State 0
State 1
State 2
State 3
State 4
State 5
State 6
State 7
State 8
State 9
State 10
State 11
State 12
State 13
State 14
State 15
State 16
State 17
State 18
State 19
State 20
cost 1012.77 s
***** Big loop 1 *****
Policy evaluation...
cost 27.25 s
Policy improvement...
cost 10.99 s
***** Big loop 2 *****
Policy evaluation...
cost 14.01 s
Policy improvement...
cost 10.85 s
***** Big loop 3 *****
Policy evaluation...
cost 9.74 s
Policy improvement...
cost 10.87 s
***** Big loop 4 *****
Policy evaluation...
cost 3.0 s
Policy improvement...
cost 10.69 s
***** Big loop 5 *****
Policy evaluation...
cost 0.87 s
Policy improvement...
cost 10.71 s
